In [0]:
/Volumes/workspace/default/pysaprk_project/menu.csv
/Volumes/workspace/default/pysaprk_project/sales.csv

In [0]:
from pyspark.sql.types import *

schema = StructType([
        StructField("product_id",IntegerType(),True),
        StructField("customer_id",StringType(),True),
        StructField("order_date",DateType(),True),
        StructField("location",StringType(),True),
        StructField("source_order",StringType(),True)
])

In [0]:
sales_df = spark.read.format("csv")\
                    .option("mode","PERMISSIVE")\
                    .schema(schema)\
                    .load("/Volumes/workspace/default/pysaprk_project/sales.csv")


display(sales_df)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

sales_df = sales_df.withColumn("order_year",year(col("order_date")))

display(sales_df)

In [0]:
sales_df = sales_df.withColumn("order_month",month(col("order_date")))

display(sales_df)

In [0]:
sales_df = sales_df.withColumn("order_quarter", quarter(col("order_date")))
display(sales_df)

In [0]:
menu_schema= StructType([
            StructField("product_id",IntegerType(),True),
            StructField("product_name",StringType(),True),
            StructField("price",IntegerType(),True)
])

In [0]:
menu_df= spark.read.format("csv")\
            .option("mode","PERMISSIVE")\
            .schema(menu_schema)\
            .load("/Volumes/workspace/default/pysaprk_project/menu.csv")


display(menu_df)

In [0]:
total_amount_spent = (sales_df.join(menu_df,"product_id").groupBy("customer_id").agg({"price":"sum"})\
                      .orderBy("customer_id"))

display(total_amount_spent)

In [0]:
total_amount_spent = (sales_df.join(menu_df,"product_id").groupBy("product_name").agg({"price":"sum"})\
                      .orderBy("product_name"))

display(total_amount_spent)

In [0]:
monthly_amount_spent = (sales_df.join(menu_df,"product_id").groupBy("order_month").agg({"price":"sum"})\
                      .orderBy("order_month"))

display(monthly_amount_spent)

In [0]:
yearly_amount_spent = (sales_df.join(menu_df,"product_id").groupBy("order_year").agg({"price":"sum"})\
                      .orderBy("order_year"))

display(yearly_amount_spent)

In [0]:
quarterly_amount_spent = (sales_df.join(menu_df,"product_id").groupBy("order_quarter").agg({"price":"sum"})\
                      .orderBy("order_quarter"))

display(quarterly_amount_spent)

In [0]:
most_purchased_product = (sales_df.join(menu_df,"product_id").groupBy("product_id","product_name")\
                                .agg(count("product_id").alias("Product_count")).orderBy(desc("Product_count"))
                            )

display(most_purchased_product)

In [0]:
most_purchased_product = (sales_df.join(menu_df,"product_id").groupBy("product_id","product_name")\
                                .agg(count("product_id").alias("Product_count")).orderBy(desc("Product_count"))\
                                    .drop("product_id")
                            )

display(most_purchased_product)

In [0]:
most_purchased_product = (sales_df.join(menu_df,"product_id").groupBy("product_id","product_name")\
                                .agg(count("product_id").alias("Product_count")).orderBy(desc("Product_count"))\
                                    .drop("product_id").limit(1)
                            )

display(most_purchased_product)

In [0]:
frequency_df = sales_df.filter(sales_df.source_order == "Restaurant").groupBy("customer_id").agg(countDistinct("order_date")).orderBy("customer_id")

display(frequency_df)

In [0]:
total_sale_country = (sales_df.join(menu_df,"product_id").groupBy("location").agg({"price":"sum"})\
                      )

display(total_sale_country)

In [0]:
total_order_source = sales_df.join(menu_df,"product_id").groupBy("source_order").agg({"price":"sum"})

display(total_order_source)